In [18]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# change for your username and password and CRUD Python module name
username = "aacuser"
password = "luna01"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
result = list(shelter.readAll());
df = pd.DataFrame.from_records(result)
df = df.iloc[:, 1:]


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso-Salvare-Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
   html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    # Also remember to include a unique identifier such as your name or date
    html.Center(html.B(html.H3('SNHU CS-340 Monticia Dunn'))),
    html.Hr(),
    # Place the HTML image tag in the line below into the app.layout code according to your design
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'display':'block', 'margin-left':'auto', 'margin-right':'auto','height':'30%', 'width':'30%'}),
    html.B(html.H4('Rescue Type:', style={'margin': '0'})),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    className='row',
        style={'display': 'flex', 'flex-direction':'column'},
        children=[
            dcc.RadioItems(
                id='rescue-type-radio-group',
               options=[
                   {'label': 'Water Rescue', 'value': 'WR'},
                   {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                   {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DR-IT'},
                   {'label': 'Reset', 'value': 'none'},
               ],
               value='none',
                labelStyle={'display': 'block'}
            )          
        ]
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
# Set up the features for your interactive data table to make it user-friendly for your client       
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('rescue-type-radio-group', 'value')])
def update_dashboard(filter_type):
### Add code to filter interactive data table with MongoDB queries
        if filter_type == 'WR':
            result = shelter.read({"animal_type":"Dog","breed":{
                "$in":["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"]
                }, "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks": {"$gte":26},
                    "age_upon_outcome_in_weeks": {"$lte":156}})
            df = pd.DataFrame(result)
            df = df.iloc[:, 1:]
        elif filter_type == 'MWR':
            result = shelter.read({"animal_type":"Dog","breed":{
                "$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog", "Siberian Husky", "Rottweiler"]
                }, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks": {"$gte":26},
                    "age_upon_outcome_in_weeks": {"$lte":156}})
            df = pd.DataFrame(result)
            df = df.iloc[:, 1:]
        elif filter_type == 'DR-IT':
            result = shelter.read({"animal_type":"Dog","breed":{
                "$in":["Doberman Pinscher","German Shepherd","Golden Retriever", "Bloodhound", "Rottweiler"]
                }, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks": {"$gte":20},
                    "age_upon_outcome_in_weeks": {"$lte":300}})
            df = pd.DataFrame(result)
            df = df.iloc[:, 1:]
        else:
            result = list(shelter.readAll());
            df = pd.DataFrame.from_records(result)
            df = df.iloc[:, 1:]
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # add code for chart of your choice (e.g. pie chart)
    df = pd.DataFrame.from_dict(viewData)
    return [
       dcc.Graph(            
           figure = px.pie(df, values=df['breed'].value_counts(), names=df['breed'].value_counts().keys(), title='Percentage of breeds available')
       )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
# Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    
    # Marker with tool tip and popup
    markers = [dl.Marker(id=str(i), position=[dff.loc[i,'location_lat'],dff.loc[i,'location_long']], children=[
                dl.Tooltip(dff.loc[i, 'name']),
                dl.Popup([
                    html.H1(dff.loc[i,'name']),
                    html.P(dff.loc[i,'breed'])
                ])
            ]) for i in range(10)]
    cluster = dl.MarkerClusterGroup(children=markers, id="cluster")
    return [
        # Austin TX is at [30.75,-97.48]
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            cluster
            
        ])
    ]


app